In [62]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import os

from ffpyplayer.player import MediaPlayer

%matplotlib inline

In [94]:
base = os.path.dirname('')
TRAIN_DIR = os.path.join(base + 'data/train')
TEST_DIR = os.path.join(base + 'data/test')
prototxt_file = os.path.join(base + 'data/deploy.prototxt.txt')
caffemodel_file = os.path.join(base + 'data/weights.caffemodel')

In [96]:
face_model = cv2.dnn.readNetFromCaffe(prototxt_file, caffemodel_file)

In [102]:
TRAIN_DIR

'data/train'

In [107]:
a = cv2.imread(os.path.join(TRAIN_DIR + '/suyun/suyun4.jpg'))
(h,w) = a.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(a, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
face_model.setInput(blob)
detector = face_model.forward()

In [121]:
detector.shape

(1, 1, 108, 7)

In [123]:
for i in range(0, detector.shape[2]):
    rect = detector[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = rect.astype("int")

    confidence = detector[0, 0, i, 2]
    if (confidence > 0.5):
        cv2.rectangle(a, (startX, startY), (endX, endY), (255, 255, 255), 2)

cv2.imshow('face', a)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
face_detector = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

In [50]:
gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
face = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60,60))
for (x, y, w, h) in face:
    cv2.rectangle(a, (x,y), (x+w, y+h), (0,255,0), 2)
cv2.imshow('face', a)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
cv2.imshow('extracted_face', a[y:y+h, x:x+h, :])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def play_video(path):
    frame_name = 'Video'
    vid = cv2.VideoCapture(path)
#     audio = MediaPlayer(path)
    
    cv2.namedWindow(frame_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(frame_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    fps = vid.get(cv2.CAP_PROP_FPS)
    print('Video fps:', fps)
    
    while vid.isOpened():
        ret, frame = vid.read()
#         audio_frame, val = audio.get_frame()
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(30,30), maxSize=(200,200))
        for (x, y, w, h) in face:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        if not ret:
            print('End of video...')
            break
        cv2.imshow(frame_name, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
#         if val != 'eof' and audio_frame is not None:
#             img, t = audio_frame
        
    vid.release()
    cv2.destroyAllWindows()

In [ ]:
play_video('data/rcpc_loveisover_mcountdown.webm')